# Tutorial: Visualizing the Silk Road Blockchain with Graphistry and Neo4j

Investigating large datasets becomes easier by directly visualizing cypher (BOLT) query results with Graphistry. This tutorial walks through querying Neo4j, visualizing the results, and additional configurations and queries.

This analysis is based on a blockchain data extract the Graphistry team performed around court proceedings from when **Carl Force**,  the key DEA agent in the Silk Road bust, was sentenced for embezzling money from **Ross Ulbricht** (Dread Pirate Roberts). We explore to how to recreate the analysis, and determine where Carl's money went after he performed the initial embezzling.

** Instructions **
* Read along the various cells
* Click the prebuilt visualizations to start them, and interact with them just like Google Maps 
* To try on your own, setup your own Neo4j instance & get a Graphistry API key, and run the data loading cells


**Further reading**

* UI Guide: https://labs.graphistry.com/graphistry/ui.html
* Python client tutorials & demos: https://github.com/graphistry/pygraphistry 
* Graphistry API Key: https://www.graphistry.com/api-request 
* Neo4j-as-a-service: http://graphstory.com 
* DEA incident: https://arstechnica.com/tech-policy/2016/08/stealing-bitcoins-with-badges-how-silk-roads-dirty-cops-got-caught/ 

## Config

In [1]:
import graphistry

GRAPHISTRY = {
    'protocol': 'http',
    'server': 'nginx'
}

NEO4J = {
    'uri': "bolt://neo4j:7687"
}

graphistry.register(bolt=NEO4J, **GRAPHISTRY)
graphistry.__version__

/opt/conda/envs/graphistry/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


'0+unknown'

## Cypher Demos

### 1a. Warmup: Visualize all $7K - $10K transactions
Try panning and zooming (same touchpad/mouse controls as Google Maps), and clicking on individual wallets and transactions.

In [2]:
g = graphistry.cypher(
    """
    MATCH (a)-[r:PAYMENT]->(b) WHERE r.USD > 7000 AND r.USD < 10000  RETURN a, r, b ORDER BY r.USD DESC
    """
)

In [3]:
g.plot()

Screenshot
![Bitcoin transactions between $7K and 10K](https://www.dropbox.com/s/kt0str2k8azs922/screenshot0.png?dl=1)

### 1b. Cleanup: Configure node and edge titles to use amount fields
* **Static config**: We can preconfigure the visualization from directly within the notebook
* **Dynamic config**: Try dynamically improving the visualization on-the-fly within the tool by 
  * Do `add histogram for...` on `edge:USD` and `point:USD_MAX`
  * Set edge/point coloring using them, and selecting a "Gradient (Spectral7 7)" blend, and toggling to reverse order (so cold to hot). 
  * For `point:USD_MAX`, toggle it to controling point size, and in the `Scene settings`,  increase the point size slider

In [13]:
g = g.bind(
    node_title='Account',
    edge_title='USD'
)

g.plot()

### 2. Look for all transactions 1-5 hops from embezzling DEA Agent Carl Force

#### 2a. Downstream
Where did most of Carl's money go? 
* Try setting up filters on `edge:USD` to separate out small vs big money flows.

In [10]:
g \
    .cypher(
        """
        match (a)-[r:PAYMENT*1..20]->(b) 
        where a.Account = $root and ALL(transfer IN r WHERE transfer.USD > $min_amount and transfer.USD < $max_amount )
        return a, r, b
        """,
        {
            'root': "Carl Force (DEA)", 
            'min_amount': 999, 
            'max_amount': 99999
        }
    ) \
    .plot() 

Screenshot:

![Carl Force's bitcoin accounts](https://www.dropbox.com/s/nh1uo4iuqvav5xm/screenshot1.png?dl=1)

#### 2b. Upstream
From where did Carl get most of his money?

In [14]:
g \
    .cypher(
        """
        match (a)-[r:PAYMENT*1..10]->(b) 
        where b.Account=$sink and ALL(transfer IN r WHERE transfer.USD > $min_amount and transfer.USD < $max_amount )
        return r, a, b
        """, 
        {
            'sink': "Carl Force (DEA)",
            'min_amount': 1999, 
            'max_amount': 99999
        }
    ) \
    .plot()

Screenshot:

![Carl Force embezzling money from the Silk Road](https://www.dropbox.com/s/qvw6s5zi1dddq78/screenshot2.png?dl=1)

## 3. Paths between Silk Road and Carl Force

In [15]:
g \
    .cypher(
        """
        match (a)-[r:PAYMENT*1..10]->(b) where a.Account=$silk and b.Account=$dea return r, a, b
        """, 
        {
            'dea': "Carl Force (DEA)",
            "silk": "Ross Ulbricht (SilkRoad)"
        }
    ) \
    .plot()

## Further Reading

* UI Guide: https://labs.graphistry.com/graphistry/ui.html
* Python client tutorials & demos: https://github.com/graphistry/pygraphistry 
* DEA incident: https://arstechnica.com/tech-policy/2016/08/stealing-bitcoins-with-badges-how-silk-roads-dirty-cops-got-caught/ 